In [0]:
!git clone https://github.com/AlexeyAB/darknet.git

Cloning into 'darknet'...
remote: Enumerating objects: 10201, done.
remote: Total 10201 (delta 0), reused 0 (delta 0), pack-reused 10201
Receiving objects: 100% (10201/10201), 10.10 MiB | 10.70 MiB/s, done.
Resolving deltas: 100% (6848/6848), done.


In [0]:
cd darknet

/content/darknet


In [0]:
!git checkout -b f154d2070aecdacd0cdfb7270e159e754ae74261

Switched to a new branch 'f154d2070aecdacd0cdfb7270e159e754ae74261'


In [0]:
!apt install libopencv-dev python-opencv ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.6-0ubuntu0.18.04.1).
libopencv-dev is already the newest version (3.2.0+dfsg-4ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  python-opencv
0 upgraded, 1 newly installed, 0 to remove and 6 not upgraded.
Need to get 535 kB of archives.
After this operation, 2,944 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 python-opencv amd64 3.2.0+dfsg-4ubuntu0.1 [535 kB]
Fetched 535 kB in 0s (4,644 kB/s)
Selecting previously unselected package python-opencv.
(Reading database ... 130911 files and directories currently installed.)
Preparing to unpack .../python-opencv_3.2.0+dfsg-4ubuntu0.1_amd64.deb ...
Unpacking python-opencv (3.2.0+dfsg-4ubuntu0.1) ...


**Set Up Cudnn** and then perform steps given below

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1TtZ3FsKs4Uzelol9sOstB8MfQ0sUZiQq'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('cudnn-10.0-linux-x64-v7.6.0.64.tgz') 

     |████████████████████████████████| 993kB 7.7MB/s 


In [0]:
! tar -zxvf cudnn-10.0-linux-x64-v7.6.0.64.tgz
! sudo cp cuda/include/cudnn.h /usr/local/cuda/include/
! sudo cp cuda/lib64/libcudnn* /usr/local/cuda/lib64/
! sudo chmod a+r /usr/local/cuda/include/cudnn.h
! sudo chmod a+r /usr/local/cuda/lib64/libcudnn*

cuda/include/cudnn.h
cuda/NVIDIA_SLA_cuDNN_Support.txt
cuda/lib64/libcudnn.so
cuda/lib64/libcudnn.so.7
cuda/lib64/libcudnn.so.7.6.0
cuda/lib64/libcudnn_static.a


In [0]:
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile

In [0]:
!make

mkdir -p obj
mkdir -p backup
mkdir -p results
chmod +x *.sh
g++ -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(mat_cv*, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:896:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
g++ -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/http_stream.cpp -o obj/http_stream.o
./src/http_stream.cpp: In member function ‘bool JSON_sende

In [0]:
!wget https://pjreddie.com/media/files/yolo.weights

--2019-05-30 05:38:19--  https://pjreddie.com/media/files/yolo.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 203934260 (194M) [application/octet-stream]
Saving to: ‘yolo.weights’

yolo.weights        100%[===================>] 194.49M  4.58MB/s    in 32s     

2019-05-30 05:38:52 (6.05 MB/s) - ‘yolo.weights’ saved [203934260/203934260]



In [0]:
%cd /content/darknet/

/content/darknet


In [0]:
!./darknet detect cfg/yolo.cfg yolo.weights data/dog.jpg

layer     filters    size              input                output
   0 conv     32  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  32 0.299 BF
   1 max          2 x 2 / 2   416 x 416 x  32   ->   208 x 208 x  32 0.006 BF
   2 conv     64  3 x 3 / 1   208 x 208 x  32   ->   208 x 208 x  64 1.595 BF
   3 max          2 x 2 / 2   208 x 208 x  64   ->   104 x 104 x  64 0.003 BF
   4 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128 1.595 BF
   5 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64 0.177 BF
   6 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128 1.595 BF
   7 max          2 x 2 / 2   104 x 104 x 128   ->    52 x  52 x 128 0.001 BF
   8 conv    256  3 x 3 / 1    52 x  52 x 128   ->    52 x  52 x 256 1.595 BF
   9 conv    128  1 x 1 / 1    52 x  52 x 256   ->    52 x  52 x 128 0.177 BF
  10 conv    256  3 x 3 / 1    52 x  52 x 128   ->    52 x  52 x 256 1.595 BF
  11 max          2 x 2 / 2    52 x  52 x 256   ->    26 x  26 x 256 0.001 

**Upload data on drive and then download here**

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1XLQFxwZMBSas5uCgcbzCHopLzy9bzs3n'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('data.tar.gz') 

In [0]:
%cd /content/

/content


In [0]:
pwd 

'/content'

In [0]:
!mkdir data

In [0]:
!tar -xzf /content/darknet/data.tar.gz data/

In [0]:
#Configure cfg file,data file,names file and train.txt and test.txt locally and then upload here

In [0]:
%cd darknet

/content/darknet


In [0]:
!wget -P /content/darknet/ https://pjreddie.com/media/files/darknet19_448.conv.23

--2019-05-30 06:32:33--  https://pjreddie.com/media/files/darknet19_448.conv.23
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79327120 (76M) [application/octet-stream]
Saving to: ‘/content/darknet/darknet19_448.conv.23’

darknet19_448.conv. 100%[===================>]  75.65M  1.29MB/s    in 55s     

2019-05-30 06:33:28 (1.38 MB/s) - ‘/content/darknet/darknet19_448.conv.23’ saved [79327120/79327120]



In [0]:
! ./darknet detector train /content/obj.data /content/yolo-obj.cfg darknet19_448.conv.23 -dont_show

yolo-obj
layer     filters    size              input                output
   0 conv     32  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  32 0.299 BF
   1 max          2 x 2 / 2   416 x 416 x  32   ->   208 x 208 x  32 0.006 BF
   2 conv     64  3 x 3 / 1   208 x 208 x  32   ->   208 x 208 x  64 1.595 BF
   3 max          2 x 2 / 2   208 x 208 x  64   ->   104 x 104 x  64 0.003 BF
   4 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128 1.595 BF
   5 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64 0.177 BF
   6 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128 1.595 BF
   7 max          2 x 2 / 2   104 x 104 x 128   ->    52 x  52 x 128 0.001 BF
   8 conv    256  3 x 3 / 1    52 x  52 x 128   ->    52 x  52 x 256 1.595 BF
   9 conv    128  1 x 1 / 1    52 x  52 x 256   ->    52 x  52 x 128 0.177 BF
  10 conv    256  3 x 3 / 1    52 x  52 x 128   ->    52 x  52 x 256 1.595 BF
  11 max          2 x 2 / 2    52 x  52 x 256   ->    26 x  26 x 2

In [0]:
!cp /yolo-obj.cfg /content/

In [0]:
%cd darknet

/content/darknet
